<a href="https://colab.research.google.com/github/zajacn/MetaVizAtlas_Tutorial_Part2/blob/main/MetaVizAtlas_Tutorial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/zajacn/MetaVizAtlas_Tutorial_Part2.git materials
! cd materials

In [ ]:
! wget -O library.metaatlas.tar.gz https://fgcz-gstore.uzh.ch/projects/p29925/library.metaatlas.tar.gz --user= --password=

In [ ]:
!tar xf library.tar.gz

In [ ]:
# activate R magic
%load_ext rpy2.ipython

In [ ]:
%%R
.libPaths('usr/local/lib/R/site-library/')

In [ ]:
%%R
library(heatmaply)
library(dplyr) # dpyr masks select from plotly
library(readr)
library(stringr)
library(tidyr)
library(tibble)
library(ggplot2)
library(ggbeeswarm)
library(pheatmap)
library(grid)
library(microbiome)
library(ape)
library(vegan)
library(useful)
library(kableExtra)
library(ggtree)
library(arrow)
library(yaml)
library(phyloseq)
library(microViz)

In [ ]:
from rpy2.rinterface_lib.callbacks import logger as rpy2_logger
import logging
rpy2_logger.setLevel(logging.ERROR)

In [ ]:
%%R
atlas_wd_folder <- "dataset/"

taxonomy_file <- paste0(atlas_wd_folder, "genomes/taxonomy/gtdb_taxonomy.tsv")
tree_file <- paste0(atlas_wd_folder, "genomes/tree/gtdbtk.bac120.nwk")
quality_file <- paste0(atlas_wd_folder, "genomes/checkm/completeness.tsv")
counts_file <- paste0(atlas_wd_folder, "genomes/counts/raw_counts_genomes.tsv")
abundance_file <- paste0(atlas_wd_folder, "genomes/counts/median_coverage_genomes.tsv")
readstats_file <- paste0(atlas_wd_folder, "stats/read_counts.tsv")
keggmodules_file <- paste0(atlas_wd_folder, "genomes/annotations/dram/kegg_modules.tsv")

In [ ]:
%%R
metadata <- read.table(file.path(atlas_wd_folder, "metadata.txt"), sep = "\t", header = TRUE, row.names = "X")
metadata <- metadata[c("Diet", "Body_weight")]
head(metadata)
group_variable <- "Diet"

We confirm that the mice on high-fat diet really put more weight on.

In [ ]:
%%R
ggplot(metadata, aes(x = Diet, y = Body_weight)) +
  geom_boxplot() +
  geom_beeswarm(cex = 5) +
  ylim(10, 35) +
  theme_minimal()

# Load Atlas output

## Taxonomy

In [ ]:
%%R
Tax <- read_tsv(taxonomy_file, show_col_types = FALSE)
print(Tax)
# create a short label for each species
Tax <- Tax %>%
  mutate(Label = ifelse(is.na(species) & is.na(genus), paste0(family, " ", user_genome), species)) %>%
  mutate(Label = ifelse(is.na(Label), paste0(genus, " ", user_genome), Label)) %>%
  mutate(species = Label, Label = NULL)

## Draw tree

In [ ]:
%%R
Tree <- ggtree::read.tree(tree_file)
Tree <- full_join(Tree, dplyr::rename(Tax, label = user_genome), by = "label")

In [ ]:
%%R
library(phytools)

tree_plot <- ggtree(Tree, layout = "daylight") # +
# geom_tiplab(aes(label=Label))




phylum_counts <- table(Tax$phylum)
phylum_to_label <- names(phylum_counts)[phylum_counts > 1]

phylum_colors <- RColorBrewer::brewer.pal(length(phylum_to_label), "Paired")
names(phylum_colors) <- phylum_to_label



for (phy in phylum_to_label) {
  phy_tax <- Tax %>% filter(phylum == phy)

  phy_mrca <- findMRCA(Tree@phylo, phy_tax$user_genome)


  tree_plot <- tree_plot +
    geom_cladelabel(
      node = phy_mrca,
      label = phy,
      fontsize = 12,
      fill = phylum_colors[phy],
      color = phylum_colors[phy],
      align = TRUE,
      offset = 0,
      offset.text = .1,
      barsize = 5,
      angle = 0,
      hjust = 0,
    )
}



tree_plot

# Genome quality

In [ ]:
%%R
genome_quality <- read_tsv(quality_file, show_col_types = FALSE) %>%
  mutate(Quality_Score = Completeness - (5 * Contamination)) %>%
  left_join(Tax, by = c("Bin Id" = "user_genome")) %>%
  mutate(Name = species) %>%
  dplyr::select(-species)

plt <- ggplot(genome_quality, aes(
  x = Contamination, y = Completeness, color = phylum,
  text = paste("Bin Id:", `Bin Id`, "\nName", Name, "\nQuality score:", Quality_Score)
)) +
  geom_point() +
  xlab("Contamination [%]") +
  ylab("Completeness [%]") +
  theme_minimal()

plt

## Relative abundance

For the relative abundance, we take the coverage over the genome, not the raw counts.
This implicitly normalizes for genome size. The coverage is calculated as the median of the coverage values calculated in 1kb blocks.


In [ ]:
%%R
D <- read_tsv(abundance_file, show_col_types = FALSE) %>%
  column_to_rownames("...1") %>%
  as.matrix()
print(topleft(D, c = 10))

# calculate relative abundance
rel_ab <- sweep(D, 1, rowSums(D), `/`)

# get most abundant genomes
counts_per_genome <- data.frame(sums = colSums(rel_ab)) %>%
  rownames_to_column(var = "Genome") %>%
  left_join(Tax, by = c("Genome" = "user_genome")) %>%
  arrange(desc(sums))


In [ ]:
%%R
ggplot(counts_per_genome %>%
  top_n(sums, n = 10), aes(x = reorder(species, -sums), y = sums)) +
  geom_col() +
  labs(x = "", y = "Abundance [rel_ab]", title = "Most abundant genomes") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 90))

### Phyloseq

Let's create a phyloseq object for further analysis.

In [ ]:
%%R
nw_tree <- ape::read.tree(tree_file)

# if you have real metadate the better

if (!exists("metadata")) {
  metadata <- data.frame(row.names = rownames(rel_ab))
  metadata$group <- "NoGroup"
}


pseq <- phyloseq(
  otu_table(rel_ab, taxa_are_rows = FALSE),
  tax_table(as.matrix(column_to_rownames(Tax, "user_genome"))),
  phy_tree(nw_tree),
  sample_data(metadata)
)

# fix taxonomy
pseq <- pseq %>%
  tax_fix()

microViz::phyloseq_validate(pseq)

### Ordination analysis

Here we use the recommended robust Centered log ratio transformation with PCA.
This corresponds to a PCoA with Aitchison distances.



In [ ]:
%%R
ordination <- pseq %>%
  tax_transform(rank = "species", trans = "rclr") %>%
  ord_calc(
    method = "auto"
  )


ordination %>%
  ord_plot(
    plot_taxa = 1:3,
    colour = "Diet", fill = "Diet",
    size = 2
  ) +
  stat_ellipse(aes(linetype = Diet, colour = Diet), linewidth = 0.3)

### Typical bar chart

In [ ]:
%%R
pseq %>% comp_barplot(tax_level = "phylum", facet_by = "Diet") +
  coord_flip() +
  theme(axis.text.y = element_blank(), axis.ticks.y = element_blank())

Heatmap

In [ ]:
%%R
cols <- distinct_palette(n = 2, add = NA)
names(cols) <- unique(samdat_tbl(pseq)$Diet)




htmp <- pseq %>%
  tax_transform("compositional", rank = "genus") %>%
  tax_filter(min_prevalence = 0.25, prev_detection_threshold = 0.001, use_counts = FALSE) %>%
  comp_heatmap(
    grid_col = NA,
    cluster_rows = FALSE, row_title = NULL,
    # row_split = 5, #seq_len(ntaxa(ps_get(pseq))),
    tax_anno = taxAnnotation(
      Prev. = anno_tax_prev(bar_width = 0.9, size = grid::unit(1, "cm"), border = F)
    ),
    sample_anno = sampleAnnotation(
      # method one
      group = anno_sample("Diet"),
      col = list(group = cols), border = TRUE
    )
  )
htmp %>% ComplexHeatmap::draw(
  annotation_legend_list = attr(htmp, "AnnoLegends")
)

# Functional annotations of genomes

The relative abundance of functional annotations per sample

The abundance is calculated as the sum of the relative abundance of all bacteria containing a function.

## Kegg modules produced by Dram

In [ ]:
%%R
kegg_modules <- read_tsv(keggmodules_file, col_select = -1)
module_names <- kegg_modules %>%
  dplyr::select(c("module", "module_name")) %>%
  distinct() %>%
  column_to_rownames("module")


print(head(kegg_modules))

In [ ]:
%%R
step_coverage_threshold <- 0.8

module_step_coverage_matrix <- pivot_wider(kegg_modules,
  id_cols = genome,
  names_from = module,
  values_from = step_coverage
) %>%
  column_to_rownames("genome") %>%
  as.matrix()

module_step_coverage_matrix <- module_step_coverage_matrix[, colSums(module_step_coverage_matrix) > 0]

setHook("grid.newpage", function() pushViewport(viewport(x = 1, y = 1, width = 0.9, height = 0.9, name = "vp", just = c("right", "top"))), action = "prepend")
hatmap <- pheatmap(module_step_coverage_matrix, show_colnames = F, show_rownames = F)
setHook("grid.newpage", NULL, "replace")
grid.text("Modules", y = -0.07, gp = gpar(fontsize = 16))
grid.text("Genomes", x = -0.07, rot = 90, gp = gpar(fontsize = 16))



module_presence_matrix <- 1 * (module_step_coverage_matrix > step_coverage_threshold)

module_presence_matrix <- module_presence_matrix[, colSums(module_presence_matrix) > 0]

hatmap

In [ ]:
%%R
d_tax <- column_to_rownames(Tax, "user_genome")



annotations <- matrix("", nrow = nrow(module_step_coverage_matrix), ncol = ncol(module_step_coverage_matrix))
colnames(annotations) <- colnames(module_step_coverage_matrix)
rownames(annotations) <- rownames(module_step_coverage_matrix)

for (genome in rownames(annotations))
{
  for (module in colnames(annotations))
  {
    annotations[genome, module] <- paste0(
      "Name: ", d_tax[genome, "species"],
      "\nPhylum: ", d_tax[genome, "phylum"],
      "\nPathway: ", module_names[module, "name"]
    )
  }
}




heatmaply(module_step_coverage_matrix,
  custom_hovertext = annotations,
  row_side_colors = list(Phylum = d_tax[rownames(module_step_coverage_matrix), "phylum"]),
  showticklabels = c(FALSE, FALSE)
)

### Module abundance

Calculate Abundance of pathways as the sum of abundance of species where a module is presence. This is equal to the matrix multiplication.


In [ ]:
%%R
stopifnot(dim(rel_ab)[2] == dim(module_presence_matrix)[1])
module_rel_ab <- as.matrix(rel_ab) %*% module_presence_matrix


setHook("grid.newpage", function() pushViewport(viewport(x = 1, y = 1, width = 0.9, height = 0.9, name = "vp", just = c("right", "top"))), action = "prepend")
pheatmap(module_rel_ab, show_rownames = F, show_colnames = F)

setHook("grid.newpage", NULL, "replace")
grid.text("Modules", y = -0.07, gp = gpar(fontsize = 16))
grid.text("Samples", x = -0.07, rot = 90, gp = gpar(fontsize = 16))

In [ ]:
%%R
annotations <- matrix("", nrow = nrow(module_rel_ab), ncol = ncol(module_rel_ab))
colnames(annotations) <- colnames(module_rel_ab)
rownames(annotations) <- rownames(module_rel_ab)
for (sample in rownames(annotations))
{
  for (module in colnames(annotations))
  {
    annotations[sample, module] <- paste0(
      "Pathway: ", module_names[module, ]
    )
  }
}




heatmaply(module_rel_ab,
  custom_hovertext = annotations,
  showticklabels = c(FALSE, FALSE),
  row_side_colors = list(Diet = metadata[rownames(module_rel_ab), "Diet"]),
  colors = "RdBu"
)

In [ ]:
%%R
abundance_per_module <- data.frame(abundance = colMeans(module_rel_ab)) %>% arrange(desc(abundance))
abundance_per_module <- cbind(abundance_per_module, module_names[rownames(abundance_per_module), ])
colnames(abundance_per_module) <- c("Average_abundance", "Description")






ggplot(abundance_per_module, aes(x = Average_abundance)) +
  geom_histogram() +
  labs(x = "Average module abundance", y = "counts") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 90))